In [1]:
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1011. Emission Mixtures/P1011.3. Analyses/P1011.3.1. Biomarker Distribution Analysis/Output')
cur_date = "040322"

library(readxl)
library(tidyverse)
library(reshape2)

# reading in files
mRNA_df = data.frame(read_excel("Input/mRNA_Data_040323.xlsx", sheet = 2))
demographics_df = data.frame(read_excel("Input/Subject_Info_031723.xlsx", sheet = 2))

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths




In [2]:
head(mRNA_df)
head(demographics_df)

,Subject_No,Subject_ID,Gene_Name,Condensate,Burn_Condition,Concentration,Time_Point,ddCT,ddCT_pslog2
,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,1,F_1,ALDH1A3,C,F,1,24,-6.6824749,4.480105
2,2,M_2,ALDH1A3,C,F,1,24,0.2560009,4.870661
3,3,M_3,ALDH1A3,C,F,1,24,0.4318636,4.879307
4,4,F_4,ALDH1A3,C,F,1,24,0.4193253,4.878692
5,6,M_6,ALDH1A3,C,F,1,24,-0.1550695,4.850246
6,1,F_1,ALDH1A3,C,F,1,72,-0.2119125,4.847400


,Original_Subject_ID,Subject_ID,Subject_No,Sex,Age,Race,Ethnicity
,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>
1,68J,F_1,1,F,18,B,NH
2,53K,M_2,2,M,32,B,NH
3,21O,M_3,3,M,34,W,H
4,63O,F_4,4,F,32,W,H
5,67L,F_5,5,F,17,W,H
6,57N,M_6,6,M,59,W,NH


Testing for statistical differences by comparing a biomarker between burn conditions (control, smoldering or flaming) to a control. (The condensate, concentration, and time point would remain consistent). 

Later adding sex in as a covariate.

Typically, normality and homogeneity of variances would be tested for first, however a non-parametric test (Wilcoxon rank sum) will be used given the small sample size (*n* = 6).

In [3]:
# the mRNA df doesn't consisently test all 3 concentrations (1,5, or 25 micrograms) or 
# time points(4,24,72) therefore we can't subset the df using a loop so we'll create separate dfs
# for each concentration and time point
split_mRNA_df = mRNA_df %>%
    group_by(Concentration, Time_Point) %>%
    group_split()

conc1_24_mRNA_df = split_mRNA_df[[1]]
conc1_72_mRNA_df = split_mRNA_df[[2]]
conc25_4_mRNA_df = split_mRNA_df[[3]]
conc25_24_mRNA_df = split_mRNA_df[[4]]
conc25_72_mRNA_df = split_mRNA_df[[5]]
conc5_24_mRNA_df = split_mRNA_df[[6]]
conc5_72_mRNA_df = split_mRNA_df[[7]]
concc_4_mRNA_df = split_mRNA_df[[8]]
concc_24_mRNA_df = split_mRNA_df[[9]]
concc_72_mRNA_df = split_mRNA_df[[10]]

In [4]:
# control samples have a concentration of NA, so they're all in a separate df
# adding them back into the other dataframes
conc1_24_mRNA_df = rbind(conc1_24_mRNA_df, concc_24_mRNA_df)
conc1_72_mRNA_df = rbind(conc1_72_mRNA_df, concc_72_mRNA_df)
conc25_4_mRNA_df = rbind(conc25_4_mRNA_df, concc_4_mRNA_df)
conc25_24_mRNA_df = rbind(conc25_24_mRNA_df, concc_24_mRNA_df)
conc25_72_mRNA_df = rbind(conc25_72_mRNA_df, concc_72_mRNA_df)
conc5_24_mRNA_df = rbind(conc5_24_mRNA_df, concc_24_mRNA_df)
conc5_72_mRNA_df = rbind(conc5_24_mRNA_df, concc_72_mRNA_df)

head(conc1_24_mRNA_df)

Subject_No,Subject_ID,Gene_Name,Condensate,Burn_Condition,Concentration,Time_Point,ddCT,ddCT_pslog2
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,F_1,ALDH1A3,C,F,1,24,-6.6824749,4.480105
2,M_2,ALDH1A3,C,F,1,24,0.2560009,4.870661
3,M_3,ALDH1A3,C,F,1,24,0.4318636,4.879307
4,F_4,ALDH1A3,C,F,1,24,0.4193253,4.878692
6,M_6,ALDH1A3,C,F,1,24,-0.1550695,4.850246
2,M_2,ALDH1A3,C,S,1,24,-0.3182245,4.842062


In [5]:
wilcoxon_rank_sumv1 = function(df){
    # """
    # Running wilcoxon rank sums after filtering for gene and condensate using a loop. 
    # Ultimately using this test to compare gene expression (control vs. flaming burn condition).

    # :param: subsetted dataframe, empty dataframe
    # :output: a dataframe containing the gene, condensate, comparison, conc, time point, stat, p value, p adj

    # """
    
    # variables that will be iterated through
    genes = unique(df$Gene_Name)
    condensates = c("C", "P")
    
    values_df = data.frame()
     # iterating through each gene, condensate, and burn condition
    for(i in 1:length(genes)){
        for(j in 1:length(condensates)){
                
            # control df
            control_df = df %>%
                filter(Gene_Name == genes[i], Condensate == "PBS")

            # flaming df
            flaming_df = df %>%
                filter(Gene_Name == genes[i], Condensate == condensates[j] , Burn_Condition == "F")

            # wilcoxon rank sum
            wilcoxon_test = wilcox.test(control_df$ddCT_pslog2, flaming_df$ddCT_pslog2)

            # contains gene, condensate, burn conditions, conc, time point, stat, and p value
            values_vector = cbind(genes[i], condensates[j], "Control vs. Flaming",
                                  unique(df$Concentration)[1], unique(df$Time_Point), 
                                  wilcoxon_test$statistic, wilcoxon_test$p.value)
            values_df = rbind(values_df, values_vector)
        }
    }

    
    # adding col names
    colnames(values_df) = c("Gene", "Condensate", "Comparison", "Concentration", "Time Point",
                            "Statistic", "P Value")
    
    # calculating padj values
    values_df$`P Adj` = p.adjust(as.numeric(as.character(values_df$`P Value`)), method = "fdr")

    
    return(values_df)
}

In [6]:
# calling fn
conc1_24_wilcox_values_v1 = wilcoxon_rank_sumv1(conc1_24_mRNA_df)
conc1_72_wilcox_values_v1 = wilcoxon_rank_sumv1(conc1_72_mRNA_df)
conc25_4_wilcox_values_v1 = wilcoxon_rank_sumv1(conc25_4_mRNA_df)
conc25_24_wilcox_values_v1 = wilcoxon_rank_sumv1(conc25_24_mRNA_df)
conc25_72_wilcox_values_v1 = wilcoxon_rank_sumv1(conc25_72_mRNA_df)
conc5_24_wilcox_values_v1 = wilcoxon_rank_sumv1(conc5_24_mRNA_df)
conc5_72_wilcox_values_v1 = wilcoxon_rank_sumv1(conc5_72_mRNA_df)

In [7]:
# creating 1 df
v1_smoldering_flaming_df = rbind(conc1_24_wilcox_values_v1, conc1_72_wilcox_values_v1, conc25_4_wilcox_values_v1,
                             conc25_24_wilcox_values_v1, conc25_72_wilcox_values_v1, conc5_24_wilcox_values_v1,
                             conc5_72_wilcox_values_v1)

In [8]:
wilcoxon_rank_sumv2 = function(df){
    # """
    # Running wilcoxon rank sums after filtering for gene and condensate using a loop. 
    # Ultimately using this test to compare gene expression (smoldering vs. flaming burn condition).

    # :param: subsetted dataframe, empty dataframe
    # :output: a dataframe containing the gene, condensate, comparison, conc, time point, stat, p value, p adj

    # """
    
    # variables that will be iterated through
    genes = unique(df$Gene_Name)
    condensates = c("C", "P")
    
    values_df = data.frame()
     # iterating through each gene, condensate, and burn condition
    for(i in 1:length(genes)){
        for(j in 1:length(condensates)){
                
            # control df
            control_df = df %>%
                filter(Gene_Name == genes[i], Condensate == condensates[j], Burn_Condition == "S")

            # flaming df
            flaming_df = df %>%
                filter(Gene_Name == genes[i], Condensate == condensates[j] , Burn_Condition == "F")

            # wilcoxon rank sum
            wilcoxon_test = wilcox.test(control_df$ddCT_pslog2, flaming_df$ddCT_pslog2)

            # contains gene, condensate, burn conditions, conc, time point, stat, and p value
            values_vector = cbind(genes[i], condensates[j], "Smoldering vs. Flaming",
                                  unique(df$Concentration)[1], unique(df$Time_Point), 
                                  wilcoxon_test$statistic, wilcoxon_test$p.value)
            values_df = rbind(values_df, values_vector)
            
        }
    }

    
    # adding col names
    colnames(values_df) = c("Gene", "Condensate", "Comparison", "Concentration", "Time Point",
                            "Statistic", "P Value")
    
    # calculating padj values
    values_df$`P Adj` = p.adjust(as.numeric(as.character(values_df$`P Value`)), method = "fdr")

    
    return(values_df)
}

In [9]:
# calling fn
conc1_24_wilcox_values_v2 = wilcoxon_rank_sumv2(conc1_24_mRNA_df)
conc1_72_wilcox_values_v2 = wilcoxon_rank_sumv2(conc1_72_mRNA_df)
conc25_4_wilcox_values_v2 = wilcoxon_rank_sumv2(conc25_4_mRNA_df)
conc25_24_wilcox_values_v2 = wilcoxon_rank_sumv2(conc25_24_mRNA_df)
conc25_72_wilcox_values_v2 = wilcoxon_rank_sumv2(conc25_72_mRNA_df)
conc5_24_wilcox_values_v2 = wilcoxon_rank_sumv2(conc5_24_mRNA_df)
conc5_72_wilcox_values_v2 = wilcoxon_rank_sumv2(conc5_72_mRNA_df)

In [10]:
# creating 1 df
v2_smoldering_flaming_df = rbind(conc1_24_wilcox_values_v2, conc1_72_wilcox_values_v2, conc25_4_wilcox_values_v2,
                             conc25_24_wilcox_values_v2, conc25_72_wilcox_values_v2, conc5_24_wilcox_values_v2,
                             conc5_72_wilcox_values_v2)

mRNA_wilcoxon_df = rbind(v1_smoldering_flaming_df, v2_smoldering_flaming_df)
head(mRNA_wilcoxon_df)

,Gene,Condensate,Comparison,Concentration,Time Point,Statistic,P Value,P Adj
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
W,ALDH1A3,C,Control vs. Flaming,1,24,25,0.00793650793650794,0.02128852
W1,ALDH1A3,P,Control vs. Flaming,1,24,30,0.00432900432900433,0.01645022
W2,CYP1B1,C,Control vs. Flaming,1,24,25,0.00793650793650794,0.02128852
W3,CYP1B1,P,Control vs. Flaming,1,24,30,0.00432900432900433,0.01645022
W4,CYP1A1,C,Control vs. Flaming,1,24,25,0.00793650793650794,0.02128852
W5,CYP1A1,P,Control vs. Flaming,1,24,30,0.00432900432900433,0.01645022


In [11]:
# exporting
write.csv(mRNA_wilcoxon_df, paste0(Output,"/", cur_date, "_mRNA_Wilcoxon_Results.csv"), row.names = FALSE)